In [ ]:
from src.lib import *
from src.transfomer import *

## Data Prepocessing

In [ ]:
with open("./data/Shakespeare.txt",'r') as f:
    text = f.read()

In [ ]:
chars = sorted(list(set(text)))

In [ ]:
vocab_size = len(chars)

In [ ]:
vocab_size

In [ ]:
str_to_idx = {char:idx for idx,char in enumerate(chars)}
idx_to_str = {idx:char for idx,char in enumerate(chars)}

In [ ]:
encode = lambda s :[str_to_idx[i] for i in s]
decode = lambda l : "".join([idx_to_str[i] for i in l ])

In [ ]:
text_encoded = encode(text)
text_encoded = torch.tensor(text_encoded)

In [ ]:
p = 0.9
n = int(p * len(text_encoded))
train_data = text_encoded[0:n]
test_data = text_encoded[n:]

In [ ]:
train_data.shape,test_data.shape

## Embedding

In [ ]:
B,T,C = 4,8,2
torch.tril(torch.rand(3,3))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = Transformer(
    vocab_size=vocab_size,
    embed_size=384,
    heads=6,
    num_layers=6,
    max_len=1000,
    device=device,
    decode_vocab=decode,
    dropout=0.2
)

In [ ]:
model.fit(train_data,batch_size=8,block_size=16,n_iter=10,print_every=1)

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.inference(context, max_token=500)[0].tolist()))